# Worst-Case Simulations

In [17]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from utils_2 import generate_uniform_random_voters, avg_util, borda_scores, true_ranking, one_round_ranking, winner_peeling_ranking, distortion, distortion_with_weights, plot_utility_curves, generate_normal_random_voters

## Generating Worst-Case Users

In [4]:
NUM_ROUNDS = 1000
NUM_VOTERS = 3
NUM_CANDIDATES = 10
CANDIDATES = np.arange(NUM_CANDIDATES)

In [10]:
CANDIDATES.shape

(10,)

In [14]:
from scipy.optimize import minimize

def f(voters, candidates, beta):
    voters_block = voters
    voters = []
    for i in range(voters_block.shape[0] // len(candidates)):
        voters.append(voters_block[i * len(candidates), i * len(candidates) + len(candidates)])

    orr = one_round_ranking(voters=voters, candidates=candidates, beta=beta)
    orr_dist = distortion(voters=voters, candidates=candidates, your_ranking=orr, return_weights=False)

    return orr_dist

In [12]:
NUM_ROUNDS = 1000
NUM_VOTERS = 3
NUM_CANDIDATES = 10
CANDIDATES = np.arange(NUM_CANDIDATES)

In [7]:
BETA = 80

In [8]:
def objective(voters):
    return -f(voters, candidates=CANDIDATES, beta=BETA)

In [16]:
res = minimize(
    fun=objective,
    x0=np.ones(NUM_VOTERS * NUM_CANDIDATES),
    bounds=[(0, 1)] * NUM_CANDIDATES * NUM_VOTERS,
    method="L-BFGS-B"
)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
beta = []
orr_dists = []
wpr_dists = []
avg_diff_squareds = []
stds = []

for i in range(NUM_ROUNDS):
    std = 2.0 * np.random.random(1)[0]
    VOTERS = generate_normal_random_voters(num_voters=NUM_VOTERS, num_candidates=NUM_CANDIDATES, std=std)
    stds.append(std)
    # VOTERS_DISTR = np.random.random(NUM_VOTERS)
    # VOTERS_DISTR /= VOTERS_DISTR.sum()
    VOTERS_DISTR = None

    # CANDIDATES_DISTR = np.random.random(NUM_CANDIDATES)
    # CANDIDATES_DISTR /= CANDIDATES_DISTR.sum()
    CANDIDATES_DISTR = None

    # BETA = np.random.random(1)[0]
    BETA = 100 * np.random.random(1)[0] + 1e-6
    # BETA = 10
    orr = one_round_ranking(voters=VOTERS, candidates=CANDIDATES, voters_distr=VOTERS_DISTR, candidates_distr=CANDIDATES_DISTR, beta=BETA)
    orr_dist, orr_weights = distortion(voters=VOTERS, candidates=CANDIDATES, your_ranking=orr, voters_distr=VOTERS_DISTR, return_weights=True)

    # weights = np.arange(NUM_CANDIDATES, 0, -1).astype(np.float64)
    # weights /= weights.sum()
    # orr_dist2 = distortion_with_weights(voters=VOTERS, candidates=CANDIDATES, your_ranking=orr, weights=weights, voters_distr=VOTERS_DISTR)

    wpr = winner_peeling_ranking(voters=VOTERS, candidates=CANDIDATES, voters_distr=VOTERS_DISTR, candidates_distr=CANDIDATES_DISTR, beta=BETA)
    wpr_dist, wpr_weights = distortion(voters=VOTERS, candidates=CANDIDATES, your_ranking=wpr, voters_distr=VOTERS_DISTR, return_weights=True)
    tr = true_ranking(voters=VOTERS, candidates=CANDIDATES, voters_distr=VOTERS_DISTR)

    avg_utils = avg_util(voters=VOTERS, candidates=CANDIDATES, voters_distr=VOTERS_DISTR)

    avg_diff_squared = 0
    for i, c1 in enumerate(CANDIDATES):
        for j, c2 in enumerate(CANDIDATES):
            avg_diff_squared += 1 / (len(CANDIDATES) * len(CANDIDATES)) * (np.abs(avg_utils[i] - avg_utils[j])) ** 3

    # print(avg_diff_squared)
    avg_diff_squareds.append(avg_diff_squared)

    beta.append(BETA)
    # orr_tr.append(np.abs(orr_dist - wpr_dist))
    orr_dists.append(orr_dist)
    wpr_dists.append(wpr_dist)

    # if (orr_dist > 1.2 or wpr_dist > 1.2) and orr_weights[0] < 1.0:
    # if (orr_dist > 1.6 or wpr_dist > 1.6):
    # if (orr_dist != wpr_dist):
    #     print(f'\nROUND {i}')
    #     print('one round ranking:', orr)
    #     print('one round distortion:',orr_dist)
    #     print('one round weights', orr_weights)

    #     # print(orr_dist2)
    #     # print(weights)

    #     print('winner peeling ranking:', wpr)
    #     print('winner peeling distortion:', wpr_dist)
    #     print('winner peeling weights', wpr_weights)

    #     for i, v in enumerate(VOTERS):
    #         print(f'voter {i}:', [v(c) for c in CANDIDATES])

    #     print('voter distribution:', VOTERS_DISTR)
    #     print('candidate distribution:', CANDIDATES_DISTR)
    #     print('beta:', BETA)

    #     tr = true_ranking(voters=VOTERS, candidates=CANDIDATES, voters_distr=VOTERS_DISTR)

    #     # max_dist_idx = 0
    #     # max_dist = 0
    #     # avg_utils = avg_util(voters=VOTERS, candidates=CANDIDATES, voters_distr=VOTERS_DISTR)
    #     # dists = []
    #     # for i in range(NUM_CANDIDATES):
    #     #     d = avg_utils[wpr[i]] / avg_utils[tr[i]]
    #     #     if d > max_dist:
    #     #         max_dist = d
    #     #         max_dist_idx = i
            
    #     print('true ranking:', tr)
    #     print('max_dist_idx:', max_dist_idx)
    #     print('max_dist:', max_dist)
    #     print('avg_util:', avg_utils)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(avg_diff_squareds, orr_dists)

cc = np.corrcoef(avg_diff_squareds, wpr_dists)[0, 1]
plt.xlabel(f'cubic moment in differences {cc}')
plt.ylabel('distortion')
plt.show()